In [5]:
import itertools

import editdistance
from nltk import ngrams

In [6]:
with open("../List_2/outputs/indexed_contents.txt", "r") as f:
    content_lines = f.readlines()

with open("../List_2/outputs/indexed_titles.txt", "r") as f:
    titles_lines = f.readlines()

In [7]:
def preprocess(word):

    new_word = ""
    final_word = ""

    s1 = {"ó": "o",
     "ę": "e",
     "ą": "a",
     "ć": "c",
     "ź": "z",
     "ż": "z"}

    s2 = {"rz": "ż",
     "ch": "h"}

    for i in range(len(word)):
        if (i+1 < len(word)) and (word[i] + word[i+1] in s2):
            new_word += s2[word[i] + word[i+1]]
        else:
            new_word += word[i]

    for i in range(len(new_word)):
        if new_word[i] in s1:
            final_word += s1[new_word[i]]
        else:
            final_word += new_word[i]

    final_word = ''.join(c[0] for c in itertools.groupby(final_word))
    return final_word

In [8]:
titles_lines = [title.strip() for title in titles_lines]

In [73]:
titles_with_brackets = [(title, idx) for idx, title in enumerate(titles_lines) if title.find('(') != -1]

In [74]:
titles_with_brackets

[('Andrzejewo (województwo mazowieckie)', 30),
 ('Alabama (ujednoznacznienie)', 72),
 ('Alaska (ujednoznacznienie)', 73),
 ('Arizona (ujednoznacznienie)', 74),
 ('Alabama (rzeka)', 79),
 ('Aidan (król Dalriady)', 93),
 ('Albion (Anglia)', 99),
 ('Athene (system operacyjny)', 124),
 ('Ada (imię)', 137),
 ('Alan Cox (informatyk)', 142),
 ('Ares (mitologia)', 145),
 ('AK (ujednoznacznienie)', 161),
 ('Aurelian (imię)', 169),
 ('Arkadiusz (cesarz bizantyński)', 172),
 ('Auto (teatr)', 192),
 ('Apollo 11 (archeologia)', 195),
 ('A (ujednoznacznienie)', 203),
 ('As (moneta)', 227),
 ('Absolwent (film)', 242),
 ('Aida (opera)', 283),
 ('Akcydens (poligrafia)', 290),
 ('Apsyda (architektura)', 292),
 ('Apollo (mitologia)', 308),
 ('ALICE (sztuczna inteligencja)', 311),
 ('Bajt (ujednoznacznienie)', 336),
 ('Buchta (żegluga)', 337),
 ('Białka (ujednoznacznienie)', 346),
 ('Beryl (pierwiastek)', 351),
 ('Bolesław Chrobry (powieść)', 372),
 ('Bałtyk (ujednoznacznienie)', 373),
 ('Biskupin (powiat

In [9]:
N_GRAMS = {title: set(map("".join, ngrams(preprocess(title), 2))) for title in titles_lines}

In [72]:
def correct_error(word):
    processed_word = preprocess(word)
    possible_words = {}
    word_ngrams = set(map("".join, ngrams(processed_word, 2)))

    for idx, (correct_word, correct_ngrams) in enumerate(N_GRAMS.items()):
        if len(word_ngrams & correct_ngrams) > 0.45 * len(word_ngrams):
            possible_words[correct_word] = (preprocess(correct_word), idx)

    min_editdist = 100
    best_word = []
    for correct_word, (processed_correct_word, idx) in possible_words.items():
        editdist = editdistance.eval(processed_correct_word, processed_word)
        if editdist == min_editdist:
            best_word.append((correct_word, idx))
        if editdist < min_editdist:
            min_editdist = editdist
            best_word = [(correct_word, idx)]

    return best_word

In [86]:
def postprocess(best_titles):

    current = []
    min_len = 1000

    # shortest
    for (title, idx) in best_titles:
        current_len = len(title)
        if len(title) < min_len:
            min_len = current_len
            current = [(title, idx)]
        elif len(title) == min_len:
            current.append((title, idx))

    best_titles = current
    best_title = None
    min_idx = 100000000

    # lowest nr
    for (title, idx) in best_titles:
        if idx < min_idx:
            min_idx = idx
            best_title = (title, idx)

    output = [best_title]

    # in brackets
    for title, idx in titles_with_brackets:
        if best_title[0] in title:
            output.append((title, idx))

    return output

In [76]:
%%time
correct_error("uniweraytrer wroclawaski")

CPU times: user 796 ms, sys: 0 ns, total: 796 ms
Wall time: 795 ms


[('Uniwersytet Wrocławski', 22546)]

In [77]:
%%time
correct_error("progamownaienie lfogiczne")

CPU times: user 777 ms, sys: 180 µs, total: 777 ms
Wall time: 778 ms


[('Programowanie logiczne', 14326)]

In [78]:
%%time
correct_error("programwowanielogiczne")

CPU times: user 797 ms, sys: 3.51 ms, total: 800 ms
Wall time: 803 ms


[('Programowanie logiczne', 14326)]

In [87]:
%%time
postprocess(correct_error("to nie jest kraej dla styartrych liddzo"))

CPU times: user 814 ms, sys: 0 ns, total: 814 ms
Wall time: 814 ms


[('To nie jest kraj dla starych ludzi', 462778),
 ('To nie jest kraj dla starych ludzi (film)', 448258),
 ('To nie jest kraj dla starych ludzi (powieść)', 462784)]